# Assigment 4


For assignment 4, we were asked to perform some analysis on a dataset that collected the tweets of 150 parlementarians of the federal Belgian parlement.
In this task we work with Graphical data, therefore we firstly had to run the Cypher environment, and upload a 'cyperhl' file. This way, we could perform queries on the database. 

We will be discussing multiple topics we investigated. 

After setting up Docker, we coud access Cypher and ended up with this interface. 

<img src="Assignment4/Screenshot 2024-05-17 140008.png" width="600" height="300"/>


## Test 1

For the first test, we decide to see what the most used words of every party are, that are being sent by the politicans themselves, and tweets that other people address to them.
To collect the correct data, we used the following Cypher query. 

\`\`\`cypher  
MATCH (u:User)-[:MEMBER_OF]->(p:Party)  
MATCH (u)-[:POSTED]->(t:Tweet)  
RETURN *  
\`\`\`


This query retrieves all users who are members of a party and have posted tweets, along with the details of those users, the parties they belong to, and the tweets they have posted. We decided to export the results of this query to a CSV file, to be able to perform text analysis. 


In [6]:
# Import necessary libraries
import pandas as pd
import json
import re
import pandas as pd
import json
import gensim.parsing.preprocessing as gsp
from collections import Counter
from stop_words import get_stop_words

In [4]:
# Import your CSV file
csv_file_path = "C:\\Users\\guest\\Downloads\\party.csv"
# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

In [5]:
# Define a dictionary with party names as keys and empty strings as values
party = {
    'N-VA': '',
    'PS': '',
    'Les Engagés': '',
    'MR': '',
    'cd&v': '',
    'PVDA-PTB': '',
    'VB': '',
    'Ecolo-Groen': '',
    'Open Vld': '',
    'ONAFH': '',
    'DéFI': '',
    'Vooruit': ''
}

# Iterate over each row in the DataFrame 'df'
for index, row in df.iterrows():
    # Access the first column of the row (containing information on the party), convert it to a JSON object
    p = row.iloc[0]
    test = json.loads(p)
    # Extract the 'ident' property from the JSON object
    partij = test["properties"]["ident"]
    
    # Access the second column of the row (containing information on the tweet), convert it to a JSON object
    t = row.iloc[1]
    tweet = json.loads(t)
    # Extract the 'full_text' property from the JSON object
    tekst = tweet["properties"]["full_text"]
    
    # Append the text of the tweet to the corresponding party in the 'party' dictionary
    party[partij] += tekst

In [ ]:
# Define a dictionary with party names as keys and empty strings as values
party = {
    'N-VA': '',
    'PS': '',
    'Les Engagés': '',
    'MR': '',
    'cd&v': '',
    'PVDA-PTB': '',
    'VB': '',
    'Ecolo-Groen': '',
    'Open Vld': '',
    'ONAFH': '',
    'DéFI': '',
    'Vooruit': ''
}

# Iterate over each row in the DataFrame 'df'
for index, row in df.iterrows():
    # Access the first column of the row, convert it to a JSON object
    p = row.iloc[0]
    test = json.loads(p)
    # Extract the 'ident' property from the JSON object
    partij = test["properties"]["ident"]
    
    # Access the second column of the row, convert it to a JSON object
    t = row.iloc[1]
    tweet = json.loads(t)
    # Extract the 'full_text' property from the JSON object
    tekst = tweet["properties"]["full_text"]
    
    # Append the text of the tweet to the corresponding party in the 'party' dictionary
    party[partij] += tekst

Now that we have all the text of the tweets that have been sent, it is time to do some preprocessing on this text. This way, we make sure only the relevant words remain, and thus we can have an accurate representation of the most common used words. 

In [8]:
# Define a list of preprocessing filters from the gensim library
# These filters will be used to clean the text data
preprocessing_filters = [
    gsp.strip_tags,  # Remove HTML tags
    gsp.strip_punctuation,  # Remove punctuation
    gsp.strip_multiple_whitespaces,  # Remove multiple whitespaces
    gsp.strip_numeric,  # Remove numeric characters
    gsp.strip_non_alphanum  # Remove non-alphanumeric characters
]

# Function to apply preprocessing filters to a text
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Remove URLs using regular expression
    text = re.sub(r'http\S+', '', text)
    
    # Remove characters immediately after '@'
    text = re.sub(r'@\S+', '', text)
    
    # Apply other preprocessing filters
    for filter_func in preprocessing_filters:
        text = filter_func(text)
    
    # Remove Dutch, French, and English stop words
    stop_words_nl = get_stop_words('dutch')
    stop_words_fr = get_stop_words('french')
    stop_words_en = get_stop_words('english')
    stop_words = set(stop_words_nl + stop_words_fr + stop_words_en)
    return ' '.join([word for word in text.split() if word not in stop_words])

# Iterate over each party in the 'party' dictionary
for parties in party:
    # Get the text associated with the party
    to_process = party[parties]
    
    # Preprocess the text
    processed = preprocess_text(to_process)
    
    # Update the text associated with the party in the 'party' dictionary
    party[parties] = processed

Now that we have all tweets of the parties and those that adressed them, we can see what the most used words are of each party. We will be representing them in a table, with between brackets the occurence of each word. 

In [12]:
# Define a dictionary to hold the party names as keys and empty strings as values
party_data = {
    'N-VA': '',
    'PS': '',
    'Les Engagés': '',
    'MR': '',
    'cd&v': '',
    'PVDA-PTB': '',
    'VB': '',
    'Ecolo-Groen': '',
    'Open Vld': '',
    'ONAFH': '',
    'DéFI': '',
    'Vooruit': ''
}

# Iterate over each party in the 'party' dictionary
for party_name in party:
    # Get the text associated with the party
    party_text = party[party_name]
    
    # Find all words in the text that are at least 5 characters long
    words = re.findall(r'\b\w{5,}\b', party_text.lower())
    
    # Count the occurrences of each word
    word_counts = Counter(words)
    
    # Initialize an empty list to hold the top 50 most common words and their counts
    top_words = []
    
    # Iterate over the 50 most common words and their counts
    for word, count in word_counts.most_common(50):
        # Format the word and count as a string
        word_info = f"{word} ({count})"
        
        # Add the formatted string to the list of top words
        top_words.append(word_info)
    
    # Update the party's data in the 'party_data' dictionary with the list of top words
    party_data[party_name] = top_words
df = pd.DataFrame(party_data,index=None)

In [13]:
df

,N-VA,PS,Les Engagés,MR,cd&v,PVDA-PTB,VB,Ecolo-Groen,Open Vld,ONAFH,DéFI,Vooruit
0,vivaldi (219),contre (128),ministre (47),merci (54),moeten (30),vivaldi (93),vlaams (354),contre (54),mensen (57),saint (14),femmes (64),mensen (57)
1,regering (172),ministre (95),enfants (47),ministre (45),mensen (27),parlement (73),belang (275),droit (54),moeten (43),josse (12),bruxelles (62),vooruit (51)
2,minister (148),guerre (92),merci (41),travail (41),minister (26),mutinerie (73),moeten (165),belgique (45),zoals (28),vlaanderen (8),liste (51),regering (38)
3,moeten (131),france (90),contre (27),nucléaire (41),werken (26),israël (73),vlaanderen (156),parlement (42),vandaag (25),middelkerke (6),contre (50),voorstel (37)
4,mensen (124),politique (84),santé (26),gauche (34),europese (23),contre (71),vlaamse (156),politique (40),regering (25),bourgmestre (5),parti (47),vandaag (35)
5,belgië (121),question (83),politique (25),politique (34),vandaag (22),livre (67),regering (132),ecolo (37),rusland (25),dedecker (4),droits (43),erfbelasting (34)
6,vandaag (116),macron (77),remboursement (24),toujours (33),nieuwe (22),gouvernement (62),beleid (124),agriculteurs (33),debat (24),grootste (4),justice (42),vlaanderen (30)
7,parlement (115),monde (76),partis (22),contre (30),resolutie (22),regering (62),mensen (110),travail (33),euthanasie (23),carrefour (4),santé (38),zoals (28)
8,vlaamse (102),après (73),begov (21),commission (30),respect (22),sociale (61),vandaag (105),monde (33),politiek (23),kandidatendag (3),politique (35),staat (27)
9,hamas (101),ukraine (73),majorité (20),après (29),voorstel (21),mensen (61),vlamingen (101),israël (32),maken (23),voorwelvaart (3),partis (35),willen (25)
